### Fonksyonlar

In [ ]:
def load_and_prep_image(filename, img_shape=64, colour_channel="grayscale"):
  #Read image
  img = tf.io.read_file(filename)
  #Decode the read file
  img= tf.image.decode_image(img)
  #Resize the image
  img= tf.image.resize(img,size=[img_shape,img_shape])
  #Scaling the image
  img=tf.image.rgb_to_grayscale(img, name=None)
  img= img/255.
  return img
def pred_and_plot(model, filename, class_names):
  
  #import the target image and pre process it
  img= load_and_prep_image(filename)

  #make predictions
  pred = model.predict(tf.expand_dims(img,axis=0))

  #Get the predicted class
  pred_class= class_names[int(tf.round(pred.argmax()))]
  #plot the image and predicted class
 ##### ? plt.title(f"Prediction: {pred_class}")
  plt.imshow(img)
  plt.axis("off")
  print(pred_class, pred)

In [ ]:
def plot_loss_curves(history):
  
  loss= history.history["loss"]
  val_loss=history.history["val_loss"]

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epo= range(len(history.history["loss"]))
  

  plt.plot(epo, loss, label="Training_Loss")
  plt.plot(epo, val_loss, label="val_Loss")
  plt.title=("Loss")
  plt.xlabel("epochs")
  plt.legend()

  plt.figure()
  plt.plot(epo, accuracy, label="Training_accuracy")
  plt.plot(epo, val_accuracy, label="val_accuracy")
  plt.title=("accuracy")
  plt.xlabel("epochs")
  plt.legend()

In [ ]:
# İmage Visualization
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random


def view_random_image(target_dir, target_class):
    #Setup Target Directory
    target_folder= target_dir+"/"+target_class
    
    #Get Random İmage
    random_image= random.sample(os.listdir(target_folder),1)
    print(random_image)
    #Read in the image and plot it using matplotlib
    img = mpimg.imread(target_folder+"/"+random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off");
    
    print(f"Image shape: {img.shape}")
    
    return img

In [ ]:
#THE CONFUSION MATRIX
#-------------------------------------------------
import itertools
from sklearn.metrics import confusion_matrix

def PrettyConfusionMatrix(y_true,y_pred,classes=None,figsize=(10,10),text_size=15 ):

    cm=confusion_matrix(y_true,y_pred)
    cm_norm=cm.astype("float")/cm.sum(axis=1)[:,np.newaxis]
    n_classes=cm.shape[0]

    #Making prettify
    fig, ax= plt.subplots(figsize=figsize)
    # Create matrix Plot
    cax=ax.matshow(cm,cmap=plt.cm.Blues)
    fig.colorbar(cax)

    #Create clases

    if classes:
        labels=classes
    else:
        labels=np.arange(cm.shape[0])

    #axis labeling
    ax.set(title="Confusion Matrix",
          xlabel="Predicted Label",
          ylabel="True Label",
          xticks=np.arange(n_classes),
          yticks=np.arange(n_classes),
          xticklabels=labels,
          yticklabels=labels)
    #Set x axis to bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

    #Adjust  label size
    ax.yaxis.label.set_size(text_size)
    ax.xaxis.label.set_size(text_size)
    ax.title.set_size(text_size+10)
    #set thresh hold for different Colours
    threshold= (cm.max()+cm.min())/2

    #Plot the text on each cell
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j,i,f"{cm[i,j]} ({cm_norm[i,j]*100:.1f}%)",
                horizontalalignment= "center",
                color="white"  if cm[i,j]>threshold else "black",
                size=15)

### Loading Datas

In [ ]:
import zipfile
zip_ref= zipfile.ZipFile("HOG_Datas_Upload.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
import os 
#walkthrough all classes
for dirpath,dirnames , filenames in os.walk("HOG_Datas_Upload"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in 'HOG_Datas_Upload'.
There are 7 directories and 0 images in 'HOG_Datas_Upload/test'.
There are 0 directories and 20 images in 'HOG_Datas_Upload/test/angry'.
There are 0 directories and 20 images in 'HOG_Datas_Upload/test/happy'.
There are 0 directories and 20 images in 'HOG_Datas_Upload/test/sad'.
There are 0 directories and 20 images in 'HOG_Datas_Upload/test/disgust'.
There are 0 directories and 20 images in 'HOG_Datas_Upload/test/fear'.
There are 0 directories and 20 images in 'HOG_Datas_Upload/test/neutral'.
There are 0 directories and 20 images in 'HOG_Datas_Upload/test/surprised'.
There are 7 directories and 0 images in 'HOG_Datas_Upload/train'.
There are 0 directories and 102 images in 'HOG_Datas_Upload/train/angry'.
There are 0 directories and 102 images in 'HOG_Datas_Upload/train/happy'.
There are 0 directories and 103 images in 'HOG_Datas_Upload/train/sad'.
There are 0 directories and 100 images in 'HOG_Datas_Upload/train/disgust'.
Ther

In [ ]:
#Setup train and test directories
train_dir="HOG_Datas_Upload/train/"
test_dir="HOG_Datas_Upload/test/"
#Lets get the class names
import pathlib
import numpy as np

data_dir= pathlib.Path(train_dir)
class_names= np.array(sorted(item.name for item in data_dir.glob("*")))
print(class_names)

['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprised']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_size= 64
#Rescale
train_datagen= ImageDataGenerator(rescale=1/255.)
test_datagen= ImageDataGenerator(rescale= 1/255.)

#load data from directories

train_data = train_datagen.flow_from_directory(train_dir,
                                                 target_size=(IMG_size,IMG_size),
                                                 batch_size=32,
                                                 class_mode="categorical",
                                               color_mode= "grayscale",
                                               shuffle=True)

test_data = test_datagen.flow_from_directory(test_dir,
                                                 target_size=(IMG_size,IMG_size),
                                                 batch_size=32,
                                                 class_mode="categorical",
                                             color_mode= "grayscale")

Found 713 images belonging to 7 classes.
Found 140 images belonging to 7 classes.


In [ ]:
#Model Libraries
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Flatten, Conv2D, MaxPool2D,BatchNormalization, Activation
from tensorflow.keras import Sequential

In [ ]:
model_17= Sequential([
    Conv2D(filters=32, kernel_size=(5,5), activation="relu",input_shape=(IMG_size,IMG_size,1)),
    BatchNormalization(name='batchnorm_1'),
    Conv2D(32,3, activation="relu"),
    MaxPool2D(),
    Conv2D(32,3, activation="relu"),
    Conv2D(32,3, activation="relu"),
    MaxPool2D(),
    Conv2D(16,3, activation="relu"),
    Conv2D(16,5, activation="relu"),
    BatchNormalization(name='batchnorm_2'),
    Conv2D(32,3, activation="relu"),
    MaxPool2D(),
    Flatten(),
    Dense(40, activation="relu"),
    Dense(7, activation="softmax")
])
#Compile the model

model_17.compile(loss= "categorical_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"],
                )

#fit the model
history_17=model_17.fit(
    train_data,
    epochs=100,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps= len(test_data)
    )


Epoch 1/100
23/23 [==============================] - 18s 286ms/step - loss: 2.2414 - accuracy: 0.2889 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 2/100
23/23 [==============================] - 5s 213ms/step - loss: 1.4021 - accuracy: 0.4614 - val_loss: 1.9426 - val_accuracy: 0.1500
Epoch 3/100
23/23 [==============================] - 5s 213ms/step - loss: 1.0050 - accuracy: 0.6185 - val_loss: 1.9438 - val_accuracy: 0.2429
Epoch 4/100
23/23 [==============================] - 5s 213ms/step - loss: 0.7734 - accuracy: 0.7251 - val_loss: 1.9444 - val_accuracy: 0.1429
Epoch 5/100
23/23 [==============================] - 5s 213ms/step - loss: 0.5714 - accuracy: 0.7966 - val_loss: 1.9466 - val_accuracy: 0.1429
Epoch 6/100
23/23 [==============================] - 5s 219ms/step - loss: 0.4460 - accuracy: 0.8415 - val_loss: 1.9589 - val_accuracy: 0.1429
Epoch 7/100
23/23 [==============================] - 5s 213ms/step - loss: 0.3212 - accuracy: 0.8934 - val_loss: 1.9760 - val_accuracy: 0.142

KeyboardInterrupt: ignored

In [ ]:
model_1= Sequential([
    Conv2D(filters=16, kernel_size=(3,3), activation="relu",input_shape=(IMG_size,IMG_size,1)),
    BatchNormalization(name='batchnorm_1'),
    Conv2D(16,3, activation="relu"),
    MaxPool2D(),
    Conv2D(16,3, activation="relu"),
    Conv2D(16,3, activation="relu"),
    MaxPool2D(),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(128, activation="relu"),
    Dense(128, activation="relu"),
    Dense(7, activation="softmax")
])
#Compile the model

model_1.compile(loss= "categorical_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"],
                )

#fit the model
history_1=model_1.fit(
    train_data,
    epochs=100,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps= len(test_data)
    )

Epoch 1/100
23/23 [==============================] - 8s 227ms/step - loss: 1.8309 - accuracy: 0.2805 - val_loss: 1.9455 - val_accuracy: 0.1429
Epoch 2/100
23/23 [==============================] - 3s 127ms/step - loss: 1.1078 - accuracy: 0.6017 - val_loss: 1.9409 - val_accuracy: 0.1643
Epoch 3/100
23/23 [==============================] - 3s 128ms/step - loss: 0.5701 - accuracy: 0.8022 - val_loss: 1.9145 - val_accuracy: 0.2929
Epoch 4/100
23/23 [==============================] - 3s 127ms/step - loss: 0.2565 - accuracy: 0.8990 - val_loss: 1.8585 - val_accuracy: 0.3714
Epoch 5/100
23/23 [==============================] - 3s 128ms/step - loss: 0.1728 - accuracy: 0.9355 - val_loss: 1.8668 - val_accuracy: 0.4000
Epoch 6/100
23/23 [==============================] - 3s 127ms/step - loss: 0.0498 - accuracy: 0.9874 - val_loss: 1.7488 - val_accuracy: 0.5143
Epoch 7/100
23/23 [==============================] - 3s 127ms/step - loss: 0.0433 - accuracy: 0.9860 - val_loss: 1.7659 - val_accuracy: 0.4929

KeyboardInterrupt: ignored

In [ ]:
model_2= Sequential([
    Conv2D(filters=16, kernel_size=(2,2), activation="relu",input_shape=(IMG_size,IMG_size,1)),
    BatchNormalization(name='batchnorm_1'),
    Conv2D(32,3, activation="relu"),
    MaxPool2D(),
    Conv2D(32,2, activation="relu"),
    Conv2D(32,2, activation="relu"),
    MaxPool2D(),
    Flatten(),
    Dense(1024, activation="relu"),
    Dense(7, activation="softmax")
])
#Compile the model

model_2.compile(loss= "categorical_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"],
                )

#fit the model
history_2=model_2.fit(
    train_data,
    epochs=50,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps= len(test_data)
    )

Epoch 1/50
23/23 [==============================] - 9s 257ms/step - loss: 3.2190 - accuracy: 0.2903 - val_loss: 1.9459 - val_accuracy: 0.1429
Epoch 2/50
23/23 [==============================] - 5s 207ms/step - loss: 1.1723 - accuracy: 0.5792 - val_loss: 1.9458 - val_accuracy: 0.1286
Epoch 3/50
23/23 [==============================] - 5s 209ms/step - loss: 0.5432 - accuracy: 0.8219 - val_loss: 1.9391 - val_accuracy: 0.2714
Epoch 4/50
23/23 [==============================] - 5s 208ms/step - loss: 0.2608 - accuracy: 0.9173 - val_loss: 1.9389 - val_accuracy: 0.2500
Epoch 5/50
23/23 [==============================] - 5s 208ms/step - loss: 0.2140 - accuracy: 0.9453 - val_loss: 1.9411 - val_accuracy: 0.2643
Epoch 6/50
23/23 [==============================] - 5s 208ms/step - loss: 0.0923 - accuracy: 0.9762 - val_loss: 1.9166 - val_accuracy: 0.3714
Epoch 7/50
23/23 [==============================] - 5s 212ms/step - loss: 0.0347 - accuracy: 0.9902 - val_loss: 1.8814 - val_accuracy: 0.4429
Epoch 

In [ ]:
model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 239, 239, 16)      80        
                                                                 
 batchnorm_1 (BatchNormaliza  (None, 239, 239, 16)     64        
 tion)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 237, 237, 32)      4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 118, 118, 32)     0         
 2D)                                                             
                                                                 
 conv2d_20 (Conv2D)          (None, 117, 117, 32)      4128      
                                                                 
 conv2d_21 (Conv2D)          (None, 116, 116, 32)     

In [ ]:
model_3= Sequential([
    Conv2D(filters=20, kernel_size=(2,2), activation="relu",input_shape=(IMG_size,IMG_size,1)),
    Conv2D(40,2, activation="relu"),
    BatchNormalization(name='batchnorm_1'),
    Conv2D(256,2, activation="relu"),
    Conv2D(256,2, activation="relu"),
    MaxPool2D(),
    Conv2D(32,3, activation="relu"),
    Conv2D(32,3, activation="relu"),
    MaxPool2D(),
    Flatten(),
    Dense(400, activation="relu"),
    Dense(400, activation="relu"),
    Dense(7, activation="softmax")
])
#Compile the model

model_3.compile(loss= "categorical_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"],
                )

#fit the model
history_3=model_3.fit(
    train_data,
    epochs=50,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps= len(test_data)
    )

Epoch 1/50
23/23 [==============================] - 7s 212ms/step - loss: 1.9015 - accuracy: 0.1950 - val_loss: 1.9428 - val_accuracy: 0.1857
Epoch 2/50
23/23 [==============================] - 4s 179ms/step - loss: 1.4337 - accuracy: 0.4811 - val_loss: 1.8748 - val_accuracy: 0.3786
Epoch 3/50
23/23 [==============================] - 4s 176ms/step - loss: 0.9376 - accuracy: 0.6578 - val_loss: 1.8503 - val_accuracy: 0.5071
Epoch 4/50
23/23 [==============================] - 4s 177ms/step - loss: 0.6302 - accuracy: 0.7644 - val_loss: 1.8537 - val_accuracy: 0.4786
Epoch 5/50
23/23 [==============================] - 4s 178ms/step - loss: 0.3929 - accuracy: 0.8597 - val_loss: 1.7967 - val_accuracy: 0.5714
Epoch 6/50
23/23 [==============================] - 4s 176ms/step - loss: 0.2001 - accuracy: 0.9299 - val_loss: 1.5383 - val_accuracy: 0.5786
Epoch 7/50
23/23 [==============================] - 4s 178ms/step - loss: 0.1938 - accuracy: 0.9299 - val_loss: 1.6936 - val_accuracy: 0.5929
Epoch 

In [ ]:
model_3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 62, 62, 20)        200       
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 31, 31, 20)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 29, 29, 50)        9050      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 14, 14, 50)       0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 9800)              0         
                                                                 
 dense_16 (Dense)            (None, 500)              